In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
sns.set(style="whitegrid")
import matplotlib.pyplot as plt
import os
from typing import Tuple
# opencv for image processing
import cv2
import tqdm
from sklearn.model_selection import train_test_split
%load_ext blackcellmagic


In [16]:
train_images= np.load('train_images.npy')
train_labels = np.load('train_labels.npy')
val_images= np.load('val_images.npy')
val_labels = np.load('val_labels.npy')
train_images.shape

(5216, 256, 256)

In [24]:
# Merge images
merged_images = np.concatenate((train_images, val_images), axis=0)
# Merge labels
merged_labels = np.concatenate((train_labels, val_labels), axis=0)

# Split the merged dataset into training and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(
    merged_images, merged_labels, test_size=0.15, random_state=42, stratify=merged_labels
)